In [65]:
from fastapi import FastAPI
import psycopg2
import os

In [83]:
from proc.func_file import *
from conllu import parse
from conllu.models import TokenList

name_file = "test"
data = read_file(f"output/{name_file}.conllu")
sentences = parse(data.read())

for i in range(len(sentences)):
    sentence = sentences[i]
    #print(sentence.metadata['sent_id'])

    try:
        conn = psycopg2.connect(
            host="localhost",
            database="ssg_conllu",
            user="postgres",
            password="postgres"
        )

        cur = conn.cursor()
        str_insert = f"INSERT INTO sentence (sent_id) VALUES (%s)"
        cur.execute(str_insert, (sentence.metadata['sent_id'],))

        conn.commit()

        for token in sentence:
            if token['form'] == None or token['form'] == '_':
                token_form = None
            else:
                token_form = token['form']

            if token['lemma'] == None or token['lemma'] == '_':
                token_lemma = None
            else:
                token_lemma = token['lemma']

            if token['xpos'] == None or token['xpos'] == '_':
                token_xpos = None
            else:
                token_xpos = token['xpos']

            if token['feats'] == None or token['feats'] == '_':
                token_feats = None
            else:
                token_feats = list(token['feats'].keys())[0]

            if token['deprel'] == None or token['deprel'] == '_':
                token_deprel = None
            else:
                token_deprel = token['deprel']

            if token['deps'] == None or token['deps'] == '_':
                token_deps = None
            else:
                token_deps = token['deps']

            if token['misc'] == None or token['misc'] == '_':
                token_misc = None
            else:
                token_misc = token['misc']

            str_insert = f"INSERT INTO token (id_sentence, id, form, lemma, upos, xpos, feats, head, deprel, deps, misc) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
            cur.execute(str_insert, (i+1, token['id'], token_form, token_lemma, token['upos'], token_xpos, token_feats, token['head'], token_deprel, token_deps, token_misc))

        conn.commit()

        #db_res = cur.fetchone()

        cur.close()
        conn.close()

        print(f"Готово {i}")
    except Exception as e:
        print(str(e))


Готово 0


In [56]:
try:
    conn = psycopg2.connect(
        host="localhost",
        database="ssg_conllu",
        user="postgres",
        password="postgres"
    )

    #form = input("Введите слово или словосочетание: ")
    form = "ОдНой по"

    cur = conn.cursor()

    cur.execute(f"SELECT upos, id_sentence FROM token WHERE LOWER(upos) LIKE LOWER('%{form}%')")

    conn.commit()

    db_res = cur.fetchall()

    if db_res == []:
        print('Ничего не найдено')
        cur.close()
        conn.close()
    else:
        with open("output/result_search.conllu", mode="w", encoding="utf-8") as file:
            cur = conn.cursor()
            for row in db_res:
                print(row[0])

                cur.execute(f"SELECT sent_id FROM sentence WHERE id_sentence = {row[1]}")
                conn.commit()
                db_res_w = cur.fetchall()
                file.writelines(f"# sent_id = {db_res_w[0][0]}\n")

                cur.execute(f"SELECT id, form, lemma, upos, xpos, feats, head, deprel, deps, misc FROM token WHERE id_sentence = {row[1]} ORDER BY id")
                conn.commit()
                db_res_w = cur.fetchall()
                for row_w in db_res_w:
                    str = ""
                    for attr in row_w:
                        if attr == None:
                            attr = '_'
                        str += f"{attr}	"
                    str = str[:-1]
                    file.writelines(f'{str}\n')
                file.writelines('\n')

            cur.close()
            conn.close()
except Exception as e:
    print(str(e))

Одной по улице бабушка строго запретила ей ходить.


In [ ]:
def is_int(s: str):
    try:
        int(s)
        return True
    except ValueError:
        return False

In [68]:
qnt = input("Введите количество слов или словосочетаний, по которым будет вестись поиск: ")
if is_int(qnt):
    int_qnt = int(qnt)
    if int_qnt > 0:
        #os.system('cls')
        for i in range(int_qnt):
            lemma = None
            form = None
            feats = None
            xpos = None
            head = None
            deprel = None
            menu = 0
            print("1. Слово")
            print("2. Слосочетание")
            menu = input(f"Введите номер пункта для {i+1}-ого слова/словосочетания: ")
            match menu:
                case "1":
                    while menu != 6:
                        print("1. Лемма")
                        print("2. Словоформа")
                        print("3. Морфологические признаки")
                        print("4. Синтаксическое отношение")
                        print("5. Посмотреть текущие ограничения")
                        print("6. Перейти к следующему слову/словосочетанию")
                        menu = input(f"Введите номер пункта для {i+1}-ого слова/словосочетания: ")

                        match menu:
                            case "1":
                                lemma = input(f"Введите лемму: ")

                            case "2":
                                form = input(f"Введите словоформу: ")

                            case "2":
                                form = input(f"Введите словоформу: ")


                case "2":
                    print("1. Точная форма")
                    print("2. Критерий")
                    print("3. Синтаксическое отношение")
                    print("4. Посмотреть текущие ограничения")
                    print("5. Перейти к следующему слову")

                case _:
                    print("Ошибка: данного пункта нет")
            while menu == 0:
                menu = input(f"Введите номер пункта для {i+1}-ого слова/словосочетания: ")
                pass
else:
    print('Ошибка: введено не натуральное число')

1. Лемма
2. Словоформа
3. Критерий
4. Морфологические признаки
5. Синтаксическое отношение
6. Перейти к следующему слову


In [69]:
строка = "Это пример предложения."
слова = строка.split()
print(слова)

['Это', 'пример', 'предложения.']


In [79]:
a = "WITH t1 AS(SELECT * FROM token WHERE LOWER(lemma) = LOWER('бабушка') AND LOWER(feats) = LOWER('%S%') AND LOWER(feats) LIKE LOWER('%ОД%') AND LOWER(feats) NOT LIKE LOWER('%НЕОД%') AND LOWER(deprel) = LOWER('пред') AND "
a[:-5]

"WITH t1 AS(SELECT * FROM token WHERE LOWER(lemma) = LOWER('бабушка') AND LOWER(feats) = LOWER('%S%') AND LOWER(feats) LIKE LOWER('%ОД%') AND LOWER(feats) NOT LIKE LOWER('%НЕОД%') AND LOWER(deprel) = LOWER('пред')"

In [7]:
a = False + True
print(a)

0


In [32]:
from conllu.models import TokenList
from conllu import parse
from proc.func_conllu import *

def rule_g_5(sentence: TokenList):
    bool_new_group = False
    for token in sentence:
        #Поиск ИНФ, который не завист от другого ИНФ
        if check_feat(token, 'ИНФ') and not check_feat(sentence[token['head'] - 1], 'ИНФ'):
            head_token = token
            last_inf_token = token
            while head_token['deprel'] == '_' and head_token['head'] != 1:
                head_token = sentence[head_token['head'] - 1]

            group_token = [head_token]
            inf_group_token = [head_token]
            bool_N = False
            while True:
                if head_token['head'] == 1:
                    break

                buf_head_token = sentence[head_token['head'] - 1]

                if buf_head_token['deprel'] in ['сочин', 'сент-соч', 'соч-союзн', 'кратн']:
                    break

                if buf_head_token['form'] == '_':
                    buf_head_token = get_head_component_sg(buf_head_token, sentence)

                if buf_head_token['id'] > head_token['id']:
                    break

                if buf_head_token['upos'] in ['VERB', 'NOUN', 'ADJ', 'ADV']:
                    if buf_head_token['upos'] in ['NOUN', 'ADJ', 'ADV']:
                        if bool_N:
                            break
                        bool_N = True
                    head_token = sentence[head_token['head'] - 1]
                    group_token.append(head_token)
                    if check_feat(buf_head_token, 'ИНФ'):
                        inf_group_token.append(buf_head_token)
                else:
                    break

            if head_token['xpos'] != 'Г5' or sentence[token['head'] - 1]['xpos'] != 'Г5' and head_token['deprel'] == '_':
                #Поиск всех зависимых ИНФ от найденного ИНФ
                bool_inf_child = True
                while bool_inf_child:
                    bool_inf_child = False
                    for token in sentence:
                        if token['head'] == last_inf_token['id']:
                            if check_feat(token, 'ИНФ'):
                                group_token.append(token)
                                inf_group_token.append(token)
                                last_inf_token = token
                                bool_inf_child = True
                                break

            bool_check_g5 = True
            if len(group_token) == 2:
                for token in group_token:
                    if not check_feat(token, 'ИНФ'):
                        bool_check_g5 = False
            if len(group_token) < 2:
                bool_check_g5 = False

            if bool_check_g5:
                #Поиск двух наборов пар связей (в первой все связи, в которых отсуствует найденный ИНФ, а во второй - присуствует)
                pairs = get_sorted_pairs(sentence)
                buf_pairs = pairs.copy()
                inf_pairs = []
                for pair in buf_pairs:
                    if last_inf_token['id'] in pair and last_inf_token['head'] not in pair:
                        inf_pairs.append(pair)
                        pairs.remove(pair)

                #Удалить все пары с ИНФ, которые нарушают проективность
                qnt_inf_pairs = len(inf_pairs)
                buf_inf_pairs = inf_pairs.copy()
                for inf_pair in buf_inf_pairs:
                    if last_inf_token['head'] > inf_pair[0] and last_inf_token['head'] < inf_pair[1]:
                        inf_pairs.remove(inf_pair)
                        continue
                    for pair in pairs:
                        if (inf_pair[0] > pair[0] and inf_pair[0] < pair[1] and (inf_pair[1] < pair[0] or inf_pair[1] > pair[1])) or (inf_pair[1] > pair[0] and inf_pair[1] < pair[1] and (inf_pair[0] < pair[0] or inf_pair[0] > pair[1])):
                            inf_pairs.remove(inf_pair)

                #Проверка, что нужно добавлять новую группу
                if qnt_inf_pairs > len(inf_pairs):
                    bool_new_group = True

                    #Дополнить набор слов, которые образуют новую группу
                    for pair in inf_pairs:
                        if pair[0] != last_inf_token['id']:
                            token = sentence[pair[0] - 1]
                        else:
                            token = sentence[pair[1] - 1]
                        group_token.append(token)
                        group_token.extend(get_group_child_token(token, sentence))

                    #Получить текст группы
                    buf_group_token = group_token.copy()
                    for token in buf_group_token:
                        if token['form'] == '_':
                            group_token.remove(token)
                            group_token.extend(get_components_sg(token, sentence))
                    group_token.sort(key=lambda token: token['id'])
                    phrase = ''
                    for token in group_token:
                        phrase += token['form'] + ' '
                    phrase = phrase[:-1]

                    group_token = buf_group_token.copy()

                    #Получить все слова, которые связаны со словами из группы
                    list_one_step_children_token = []
                    for token in group_token:
                        list_one_step_children_token.extend(get_group_one_step_child_token(token, sentence))

                    #Добавить новую группу и изменить связи между элементами ССГ
                    for token in list_one_step_children_token:
                        if token not in group_token:
                            token['head'] = len(sentence) + 1
                    sentence.append(create_sg(len(sentence) + 1, phrase, 'Г5', head_token['head'], head_token['deprel']))
                    head_token['deprel'] = '_'
                    head_token['head'] = len(sentence)

    return bool_new_group

In [112]:
def rule_g_6(sentence: TokenList):
    bool_new_group = False
    for token in sentence:
        #поиск ГЛ, который не является ИНФ
        if token['upos'] == 'VERB' and not check_feat(token, 'ИНФ'):
            head_token = token
            while head_token['deprel'] == '_' and head_token['head'] != 1:
                head_token = sentence[head_token['head'] - 1]

            if head_token['xpos'] != 'Г6' or sentence[token['head'] - 1]['xpos'] != 'Г6' and head_token['deprel'] == '_':
                #поиск ИНФ, который зависит от найденного слова
                for token in get_group_one_step_child_token(head_token, sentence):
                    buf_token = token
                    if buf_token['form'] == '_':
                        buf_token = get_head_component_sg(buf_token, sentence)
                    if buf_token['upos'] in ['PRON', 'NOUN', 'ADJ', 'ADV']:
                        noun_token = token

                        #Поиск двух наборов пар связей (в первой все связи, в которых отсуствует найденный СУЩ, а во второй - присуствует)
                        pairs = get_sorted_pairs(sentence)
                        noun_pairs = []
                        buf_pairs = pairs.copy()
                        if noun_token['form'] == '_':
                            id_noun_token = get_head_component_sg(noun_token, sentence)['id']
                            for token in sentence:
                                if token['head'] == id_noun_token:
                                    id_child_token = token['id']
                                    break
                            for pair in buf_pairs:
                                if id_noun_token in pair and get_id_head_token(noun_token, sentence) not in pair and id_child_token not in pair:
                                    noun_pairs.append(pair)
                                    pairs.remove(pair)
                        else:
                            id_noun_token = noun_token['id']
                            for pair in buf_pairs:
                                if id_noun_token in pair and get_id_head_token(noun_token, sentence) not in pair:
                                    noun_pairs.append(pair)
                                    pairs.remove(pair)

                        #Проверить, что существуют пары с ИНФ
                        qnt_noun_pairs = len(noun_pairs)
                        if qnt_noun_pairs > 0:

                            #Удалить все пары с ИНФ, которые нарушают проективность
                            buf_noun_pairs = noun_pairs.copy()
                            for noun_pair in buf_noun_pairs:
                                if get_id_head_token(noun_token, sentence) > noun_pair[0] and get_id_head_token(noun_token, sentence) < noun_pair[1]:
                                    noun_pairs.remove(noun_pair)
                                    continue
                                for pair in pairs:
                                    if (noun_pair[0] > pair[0] and noun_pair[0] < pair[1] and (noun_pair[1] < pair[0] or noun_pair[1] > pair[1])) or (noun_pair[1] > pair[0] and noun_pair[1] < pair[1] and (noun_pair[0] < pair[0] or noun_pair[0] > pair[1])):
                                        noun_pairs.remove(noun_pair)

                            #Проверка, что нужно добавлять новую группу
                            if qnt_noun_pairs > len(noun_pairs):
                                bool_new_group = True

                                #Получить набор слов, которые образуют новую группу
                                group_token = [head_token, noun_token]
                                for pair in noun_pairs:
                                    if pair[0] != noun_token['id']:
                                        token = sentence[pair[0] - 1]
                                    else:
                                        token = sentence[pair[1] - 1]
                                    group_token.append(token)
                                    group_token.extend(get_group_child_token(token, sentence))

                                #Получить словосочетание
                                buf_group_token = group_token.copy()
                                for token in buf_group_token:
                                    if token['form'] == '_':
                                        group_token.remove(token)
                                        group_token.extend(get_components_sg(token, sentence))
                                group_token.sort(key=lambda token: token['id'])
                                phrase = ''
                                for token in group_token:
                                    phrase += token['form'] + ' '
                                phrase = phrase[:-1]
                                group_token = buf_group_token.copy()

                                #Получить все слова, которые связаны со словами из группы
                                group_one_step_child_token = []
                                for token in group_token:
                                    group_one_step_child_token.extend(get_group_one_step_child_token(token, sentence))

                                #Добавить новую группу и изменить связи между элементами ССГ
                                for token in group_one_step_child_token:
                                    if token not in group_token:
                                        token['head'] = len(sentence) + 1
                                sentence.append(create_sg(len(sentence) + 1, phrase, 'Г6', head_token['head'], head_token['deprel']))
                                head_token['deprel'] = '_'
                                head_token['head'] = len(sentence)

        #Поиск ИНФ, который не завист от другого ИНФ
        if check_feat(token, 'ИНФ') and not check_feat(sentence[token['head'] - 1], 'ИНФ'):
            head_token = token
            last_inf_token = token
            while head_token['deprel'] == '_' and head_token['head'] != 1:
                head_token = sentence[head_token['head'] - 1]

            group_token = [head_token]
            inf_group_token = [head_token]
            bool_N = False
            while True:
                if head_token['head'] == 1:
                    break

                buf_head_token = sentence[head_token['head'] - 1]

                if buf_head_token['deprel'] in ['сочин', 'сент-соч', 'соч-союзн', 'кратн']:
                    break

                if buf_head_token['form'] == '_':
                    buf_head_token = get_head_component_sg(buf_head_token, sentence)

                if buf_head_token['id'] > head_token['id']:
                    break

                if buf_head_token['upos'] in ['VERB', 'NOUN', 'ADJ', 'ADV']:
                    if buf_head_token['upos'] in ['NOUN', 'ADJ', 'ADV']:
                        if bool_N:
                            break
                        bool_N = True
                    head_token = sentence[head_token['head'] - 1]
                    group_token.append(head_token)
                else:
                    break

            if head_token['xpos'] != 'Г6' or sentence[token['head'] - 1]['xpos'] != 'Г6' and head_token['deprel'] == '_':
                #Поиск всех зависимых ИНФ от найденного ИНФ
                bool_inf_child = True
                while bool_inf_child:
                    bool_inf_child = False
                    for token in sentence:
                        if token['head'] == last_inf_token['id']:
                            if check_feat(token, 'ИНФ'):
                                group_token.append(token)
                                inf_group_token.append(token)
                                last_inf_token = token
                                bool_inf_child = True
                                break

                for noun_token in get_group_child_token(last_inf_token, sentence):
                    bool_noun = False
                    if noun_token['form'] == '_':
                        if get_head_component_sg(noun_token, sentence)['upos'] in ['NOUN', 'ADJ', 'ADV']:
                            bool_noun = True
                    else:
                        if noun_token['upos'] in ['NOUN', 'ADJ', 'ADV']:
                            bool_noun = True

                    if bool_noun:
                        group_token.append(noun_token)
                        #Поиск двух наборов пар связей (в первой все связи, в которых отсуствует найденный СУЩ, а во второй - присуствует)
                        pairs = get_sorted_pairs(sentence)
                        buf_pairs = pairs.copy()
                        noun_pairs = []
                        for pair in buf_pairs:
                            if noun_token['id'] in pair and noun_token['head'] not in pair:
                                noun_pairs.append(pair)
                                pairs.remove(pair)

                        #Удалить все пары с СУЩ, которые нарушают проективность
                        qnt_noun_pairs = len(noun_pairs)
                        buf_noun_pairs = noun_pairs.copy()
                        for noun_pair in buf_noun_pairs:
                            if noun_token['head'] > noun_pair[0] and noun_token['head'] < noun_pair[1]:
                                noun_pairs.remove(noun_pair)
                                continue
                            for pair in pairs:
                                if (noun_pair[0] > pair[0] and noun_pair[0] < pair[1] and (noun_pair[1] < pair[0] or noun_pair[1] > pair[1])) or (noun_pair[1] > pair[0] and noun_pair[1] < pair[1] and (noun_pair[0] < pair[0] or noun_pair[0] > pair[1])):
                                    noun_pairs.remove(noun_pair)

                        #Проверка, что нужно добавлять новую группу
                        if qnt_noun_pairs > len(noun_pairs):
                            bool_new_group = True

                            #Дополнить набор слов, которые образуют новую группу
                            for pair in noun_pairs:
                                if pair[0] != noun_token['id']:
                                    token = sentence[pair[0] - 1]
                                else:
                                    token = sentence[pair[1] - 1]
                                group_token.append(token)
                                group_token.extend(get_group_child_token(token, sentence))

                            #Получить текст группы
                            buf_group_token = group_token.copy()
                            for token in buf_group_token:
                                if token['form'] == '_':
                                    group_token.remove(token)
                                    group_token.extend(get_components_sg(token, sentence))
                            group_token.sort(key=lambda token: token['id'])
                            phrase = ''
                            for token in group_token:
                                phrase += token['form'] + ' '
                            phrase = phrase[:-1]
                            group_token = buf_group_token.copy()

                            #Получить все слова, которые связаны со словами из группы
                            list_one_step_children_token = []
                            for token in group_token:
                                list_one_step_children_token.extend(get_group_one_step_child_token(token, sentence))

                            #Добавить новую группу и изменить связи между элементами ССГ
                            for token in list_one_step_children_token:
                                if token not in group_token:
                                    token['head'] = len(sentence) + 1
                            sentence.append(create_sg(len(sentence) + 1, phrase, 'Г6', head_token['head'], head_token['deprel']))
                            head_token['deprel'] = '_'
                            head_token['head'] = len(sentence)

    return bool_new_group

In [113]:
data = open("input/test.conllu", mode="r", encoding="utf-8")

sentences = parse(data.read())

bool_new_group = False
for sentence in sentences:
    add_root_token(sentence)

    bool_new_group += rule_g_6(sentence)

with open("output/test.conllu", mode="w", encoding="utf-8") as f:
    f.writelines([sentence.serialize() + "\n" for sentence in sentences])

In [85]:
data = open("input/test.conllu", mode="r", encoding="utf-8")

sentences = parse(data.read())

bool_new_group = False
sentence = sentences[6]
get_group_child_token(sentence[18], sentence)

IndexError: list index out of range

In [29]:
len(get_group_child_token(sentence[18], sentence))

0

In [34]:
sentence[18]

{'id': 19,
 'form': 'в',
 'lemma': 'В',
 'upos': 'ADP',
 'xpos': 'PR',
 'feats': {'PR': ''},
 'head': 18,
 'deprel': '2-компл',
 'deps': None,
 'misc': None}

In [104]:
if [2, 3, 4, 5] not in [3, 7]:
    print('123')

123


In [135]:
import json

with open("wordbook.json", "r", encoding="utf-8") as json_file:
    wb = json.load(json_file)

for i in range(len(wb['words'])):
    print(wb['words'][i]['lemma'])


талант
обыск
спать
ранить
власть


In [2]:
import json

from conllu.models import TokenList
from conllu import parse
from proc.func_conllu import *

def rule_g_1(sentence: TokenList):
    for head_token in sentence:
        bool_new_group = False
        magn_token = None
        if (head_token['xpos'] != 'Г1' or sentence[head_token['head'] - 1]['xpos'] != 'Г1' and head_token['deprel'] == '_') and head_token['id'] != 1:
            if head_token['form'] == '_':
                group_component_sg = get_components_sg(head_token, sentence)
                if len(group_component_sg) == 2:
                    for token in group_component_sg:
                        if token['head'] == head_token['id'] and token['deprel'] == '_':
                            lemma = token['lemma']
                else:
                    lemma = get_head_component_sg(head_token, sentence)['lemma']
            else:
                lemma = head_token['lemma']
            group_child_token = get_group_child_token(head_token, sentence)
            if len(group_child_token) > 0:
                with open("wordbook.json", "r", encoding="utf-8") as json_file:
                    wordbook = json.load(json_file)
                    for word in wordbook['words']:
                        if word['lemma'] == lemma.lower():
                            for child_token in group_child_token:
                                for magn in word['magn']:
                                    if magn == child_token['lemma'].lower():
                                        bool_new_group = True
                                        #print('1', bool_new_group)
                                        magn_token = child_token.copy()
                                        break
            if bool_new_group:
                group_token = [head_token]
                for token in get_group_child_token(head_token, sentence):
                    if token['deprel'] not in ['сравнит', 'сочин', 'сент-соч', 'соч-союзн', 'кратн']:
                        group_token.append(token)
                buf_group_token = group_token.copy()
                for token in buf_group_token:
                    if token['form'] == '_':
                        group_token.remove(token)
                        group_token.extend(get_components_sg(token, sentence))
                group_token.sort(key=lambda token: token['id'])
                phrase = ''
                for token in group_token:
                    phrase += token['form'] + ' '
                phrase = phrase[:-1]
                group_token = buf_group_token.copy()

                #Добавить новую группу и изменить связи между элементами ССГ
                for token in group_token:
                    if token not in group_token:
                        token['head'] = len(sentence) + 1
                sentence.append(create_sg(len(sentence) + 1, phrase, 'Г1', head_token['head'], head_token['deprel']))
                head_token['deprel'] = '_'
                head_token['head'] = len(sentence)

    return bool_new_group

In [22]:
data = open("conllu/tree/test.conllu", mode="r", encoding="utf-8")

sentences = parse(data.read())

bool_new_group = False
for sentence in sentences:
    add_root_token(sentence)

    bool_new_group += rule_g_1(sentence)

with open("conllu/ssg/test.conllu", mode="w", encoding="utf-8") as f:
    f.writelines([sentence.serialize() + "\n" for sentence in sentences])

In [4]:
data = open("conllu/tree/test.conllu", mode="r", encoding="utf-8")

sentences = parse(data.read())

bool_new_group = False
sentence = sentences[0]
sentence[0]

{'id': 1,
 'form': 'Сделано',
 'lemma': 'СДЕЛАТЬ',
 'upos': 'VERB',
 'xpos': None,
 'feats': None,
 'head': 0,
 'deprel': '_',
 'deps': None,
 'misc': None}

In [60]:
def _get_group_one_step_child_token(head_token: Token, sentence: TokenList) -> list:
    group_one_step_child_token = []
    for token in sentence:
        if token['head'] == head_token['id'] and token['deprel'] != '_':
            group_one_step_child_token.append(token)
    return group_one_step_child_token

In [61]:
def get_token_for_group(token: Token, sentence: TokenList):
    if token['deprel'] not in ['сравнит', 'сочин', 'сент-соч', 'соч-союзн', 'кратн']:
        group_child_token = [token]
        for child_token in _get_group_one_step_child_token(token, sentence):
            group_child_token.extend(get_token_for_group(child_token, sentence))
        return group_child_token
    else:
        return []

In [67]:
data = open("conllu/tree/test.conllu", mode="r", encoding="utf-8")

sentences = parse(data.read())

bool_new_group = False
sentence = sentences[0]

sentence[2]['deps'][0]

'опред'

In [44]:
if [] == None:
    print(123)
else:
    print(321)

321
